In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv

In [ ]:
def get_pos_vel(itime,pos_true):
    "Return measured position, true velocity and true position"
    w=np.random.normal(0,np.sqrt(10)) # w :system noise
    v=np.random.normal(0,np.sqrt(10)) # v: measurement noise
    
    vel_true=80 + w
    if itime==0:
        pos_true=pos_true
    else:
        pos_true=pos_true+vel_true*dt
    z_pos_meas=pos_true+v #z_pos_meas : mesaured position (observavle)
    return z_pos_meas,vel_true,pos_true